# Day 11

https://adventofcode.com/2022/day/11

## Part 1

In [1]:
test_result = 13
test_data = """\
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""
test_data

'Monkey 0:\n  Starting items: 79, 98\n  Operation: new = old * 19\n  Test: divisible by 23\n    If true: throw to monkey 2\n    If false: throw to monkey 3\n\nMonkey 1:\n  Starting items: 54, 65, 75, 74\n  Operation: new = old + 6\n  Test: divisible by 19\n    If true: throw to monkey 2\n    If false: throw to monkey 0\n\nMonkey 2:\n  Starting items: 79, 60, 97\n  Operation: new = old * old\n  Test: divisible by 13\n    If true: throw to monkey 1\n    If false: throw to monkey 3\n\nMonkey 3:\n  Starting items: 74\n  Operation: new = old + 3\n  Test: divisible by 17\n    If true: throw to monkey 0\n    If false: throw to monkey 1'

In [2]:
from dataclasses import dataclass
from typing import List, Callable

@dataclass
class Monkey:
    items: List[int]
    operation: Callable[[int], int]
    test_divisor: int
    targets: dict
    items_count: int = 0

def func_maker(operator: str, parameter: str):
    if operator == "*":
        if parameter == "old":
            return lambda x: x * x
        else:
            parameter = int(parameter)
            return lambda x: x * parameter
    elif operator == "+":
        parameter = int(parameter)
        return lambda x: x + parameter
    raise ValueError(f"Unknown operator {operator}")

def import_data(data: str, verbose=False):
    monkey_data = data.split("\n\n")
    monkeys = {}
    for i, monkey in enumerate(monkey_data):
        if verbose:
            print(f"Monkey {i}")    
        lines = monkey.split("\n")

        # start items (Starting items: 79, 98)
        line = lines[1].split(":")
        start_items = [int(_x) for _x in line[1].split(",")]
        if verbose:
            print(f"\tStart Items: {start_items}")

        # Operation: new = old * 19
        line = lines[2].split(" ")    
        parameter = line[-1]
        operator = line[-2]
        operation = func_maker(operator, parameter)
        if verbose:
            print(f"\tOperation: new = old {operator} {parameter})")

        # test func (Test: divisible by 23)
        line = lines[3].split(" ")
        test_divisor = int(line[-1])
        if verbose:
            print(f"\tTest Func: x % {test_divisor} == 0")

        # Target according to test result
        # If true: throw to monkey 0
        # If false: throw to monkey 1
        targets = {}
        for idx, key in zip([4, 5], [True, False]):
            line = lines[idx].split(" ")
            targets[key] = int(line[-1])
        if verbose:
            print(f"\tTargets: {targets}")

        #monkeys[i] = dict(
        #    items_start = start_items,
        #    operation = operation,         
        #    test_func = test_func,
        #    target = target, 
        #    items_count = 0
        #)
        monkeys[i] = Monkey(
            items=start_items, 
            operation=operation, 
            test_divisor=test_divisor, 
            targets=targets
        )
    return monkeys

import_data(test_data, True)

Monkey 0
	Start Items: [79, 98]
	Operation: new = old * 19)
	Test Func: x % 23 == 0
	Targets: {True: 2, False: 3}
Monkey 1
	Start Items: [54, 65, 75, 74]
	Operation: new = old + 6)
	Test Func: x % 19 == 0
	Targets: {True: 2, False: 0}
Monkey 2
	Start Items: [79, 60, 97]
	Operation: new = old * old)
	Test Func: x % 13 == 0
	Targets: {True: 1, False: 3}
Monkey 3
	Start Items: [74]
	Operation: new = old + 3)
	Test Func: x % 17 == 0
	Targets: {True: 0, False: 1}


{0: Monkey(items=[79, 98], operation=<function func_maker.<locals>.<lambda> at 0x7f26c5da95a0>, test_divisor=23, targets={True: 2, False: 3}, items_count=0),
 1: Monkey(items=[54, 65, 75, 74], operation=<function func_maker.<locals>.<lambda> at 0x7f26c5da9630>, test_divisor=19, targets={True: 2, False: 0}, items_count=0),
 2: Monkey(items=[79, 60, 97], operation=<function func_maker.<locals>.<lambda> at 0x7f26c5da96c0>, test_divisor=13, targets={True: 1, False: 3}, items_count=0),
 3: Monkey(items=[74], operation=<function func_maker.<locals>.<lambda> at 0x7f26c5da9750>, test_divisor=17, targets={True: 0, False: 1}, items_count=0)}

In [3]:
from math import lcm

def simulate_round(monkeys: dict, part=1, verbose=False):
    # the trick in part 2 is to reduce the worry level to the remainder 
    # to the least common multiplier of all test deviders...
    # otherwise it takes literally forever to compute :(
    # https://github.com/ephemient/aoc2022/blob/main/py/aoc2022/day11.py
    test_divs = (_m.test_divisor for _m in monkeys.values())
    base = lcm(*test_divs)
    
    for i, monkey_curr in monkeys.items():
        if verbose:
            print(f"\nCurrent Monkey: {i}")
        operation = monkey_curr.operation
        test_devisor = monkey_curr.test_divisor
        target = monkey_curr.targets
        for worry in monkey_curr.items:
            monkey_curr.items_count += 1
            if verbose:
                print(f"\tStart Item: {worry}")
            worry = operation(worry)
            if part == 1:
                worry //= 3
            else:
                worry = worry % base
            if verbose:
                print(f"\tAfter Inspection: {worry}")            
            test_result = worry % test_devisor == 0
            if verbose:
                print(f"\tTest Result: {test_result}")
            monkey_target = target[test_result]
            if verbose:
                print(f"\tThrowing to Monkey: {monkey_target}")
            monkeys[monkey_target].items.append(worry)
        monkey_curr.items = []
    
    return monkeys

def solution1(data, rounds=20, verbose=False):
    monkeys = import_data(data)

    for _round in range(rounds):
        monkeys = simulate_round(monkeys)
        if verbose:
            print(f"Round {_round+1}")
            for i, monkey_curr in monkeys.items():
                print(f"  Monkey {i}:")
                print(f"\tholds items: {monkey_curr['items_start']}")
                print(f"\thandled items: {monkey_curr['items_count']}")
    items_handled = [_m.items_count for _m in monkeys.values()]
    items_handled.sort()
    print(items_handled)
    sol = items_handled[-1] * items_handled[-2]
    print(sol)
    return sol
assert solution1(test_data) == 10605
print("test passed")

[7, 95, 101, 105]
10605
test passed


In [4]:
with open("day11.txt") as f:
    inp_data = f.read()

assert solution1(inp_data) == 99840
print("test passed")

[9, 34, 70, 167, 295, 311, 312, 320]
99840
test passed


## Part 2

In [5]:
monkeys = import_data(test_data)
rounds = 100000
verbose = True
for _round in range(rounds):
    monkeys = simulate_round(monkeys, part=2)
    if verbose and (_round + 1) in [1, 20, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 9000, 10000]:
        print(f"Round {_round+1}")
        for i, monkey_curr in monkeys.items():
            print(f"Monkey {i}: {monkey_curr.items_count}")
items_handled = [_m.items_count for _m in monkeys.values()]
items_handled.sort()
print(items_handled)
sol = items_handled[-1] * items_handled[-2]
print(sol)

Round 1
Monkey 0: 2
Monkey 1: 4
Monkey 2: 3
Monkey 3: 6
Round 20
Monkey 0: 99
Monkey 1: 97
Monkey 2: 8
Monkey 3: 103
Round 1000
Monkey 0: 5204
Monkey 1: 4792
Monkey 2: 199
Monkey 3: 5192
Round 2000
Monkey 0: 10419
Monkey 1: 9577
Monkey 2: 392
Monkey 3: 10391
Round 3000
Monkey 0: 15638
Monkey 1: 14358
Monkey 2: 587
Monkey 3: 15593
Round 4000
Monkey 0: 20858
Monkey 1: 19138
Monkey 2: 780
Monkey 3: 20797
Round 5000
Monkey 0: 26075
Monkey 1: 23921
Monkey 2: 974
Monkey 3: 26000
Round 6000
Monkey 0: 31294
Monkey 1: 28702
Monkey 2: 1165
Monkey 3: 31204
Round 7000
Monkey 0: 36508
Monkey 1: 33488
Monkey 2: 1360
Monkey 3: 36400
Round 9000
Monkey 0: 46945
Monkey 1: 43051
Monkey 2: 1746
Monkey 3: 46807
Round 10000
Monkey 0: 52166
Monkey 1: 47830
Monkey 2: 1938
Monkey 3: 52013
[19332, 478243, 520194, 521753]
271412780082


In [6]:
def solution2(data, rounds=10000, verbose=False):
    monkeys = import_data(data)
    for _round in range(rounds):
        monkeys = simulate_round(monkeys, part=2)
        if verbose:
            print(f"Round {_round+1}")
            for i, monkey_curr in monkeys.items():
                print(f"  Monkey {i}:")
                print(f"\tholds items: {monkey_curr['items_start']}")
                print(f"\thandled items: {monkey_curr['items_count']}")
    items_handled = [_m.items_count for _m in monkeys.values()]
    items_handled.sort()
    print(items_handled)
    sol = items_handled[-1] * items_handled[-2]
    print(sol)
    return sol
    

assert solution2(test_data) == 2713310158
print("test passed")

[1938, 47830, 52013, 52166]
2713310158
test passed


In [7]:
sol2 = solution2(inp_data)
assert sol2 == 20683044837
print("test passed")

[21064, 26874, 48933, 60049, 125646, 128049, 142623, 145019]
20683044837
test passed
